# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("../output_data/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Saint-Philippe,0,RE,04/24/2021,73,-21.3585,55.7679,79.0,26.46
1,1,Narsaq,90,GL,04/24/2021,32,60.9167,-46.0500,51.8,34.52
2,2,Faya,40,SA,04/24/2021,59,18.3851,42.4509,64.4,3.53
3,3,Jamestown,90,US,04/24/2021,53,42.0970,-79.2353,48.2,5.75
4,4,Fort Nelson,20,CA,04/24/2021,33,58.8053,-122.7002,41.0,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
#Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [5]:
#Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create & add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
107,107,Cabedelo,0,BR,04/24/2021,100,-6.9811,-34.8339,73.40,4.61
137,137,Conde,0,BR,04/24/2021,100,-7.2597,-34.9075,73.40,4.61
202,202,Beroroha,0,MG,04/24/2021,57,-21.6667,45.1667,78.08,6.58
260,260,Oussouye,0,SN,04/24/2021,94,12.4850,-16.5469,71.60,4.61
297,297,Pitimbu,0,BR,04/24/2021,100,-7.4706,-34.8086,73.40,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = ideal_weather.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
107,Cabedelo,BR,-6.9811,-34.8339,
137,Conde,BR,-7.2597,-34.9075,
202,Beroroha,MG,-21.6667,45.1667,
260,Oussouye,SN,12.4850,-16.5469,
297,Pitimbu,BR,-7.4706,-34.8086,
448,Sakakah,SA,29.9697,40.2064,
573,Kingaroy,AU,-26.5333,151.8333,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 107: Cabedelo.
Missing field/result... skipping.
------------
Retrieving Results for Index 137: Conde.
Missing field/result... skipping.
------------
Retrieving Results for Index 202: Beroroha.
Missing field/result... skipping.
------------
Retrieving Results for Index 260: Oussouye.
Missing field/result... skipping.
------------
Retrieving Results for Index 297: Pitimbu.
Missing field/result... skipping.
------------
Retrieving Results for Index 448: Sakakah.
Missing field/result... skipping.
------------
Retrieving Results for Index 573: Kingaroy.
Missing field/result... skipping.
------------
-------End of Search-------


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


# Display figure
